In [ ]:
from platform import python_version
print(python_version())

## 1) Comparing Default Cutoff x BCA: TP, FP, TN, FN

  - Given the default enriched table (tab1)
  - Given the BCA enriched Table, removing the default pathways = extra pathways (tab2)

### TP and FP
   - tab1
     - Gemini Yes --> TP
     - Gemini False --> FP
    
### TN and FN
   - tab2
     - Gemini Yes --> FN
     - Gemini False --> TN 

## 2) Comparing G0 x G1 pathways: TP, FP, TN, FN

  - Given the G0 table (tab1)
  - Given the G1 table(tab2)

### TP and FP
   - tab1
     - Gemini Yes --> TP
     - Gemini False --> FP
    
### TN and FN
   - tab2
     - Gemini Yes --> FN
     - Gemini False --> TN

In [ ]:
import os, sys, pickle

import numpy as np
import pandas as pd
pd.set_option('display.width', 100)
pd.set_option('max_colwidth', 80)
import yaml

import seaborn as sns
sns.set_context("notebook", font_scale=1.4)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

sys.path.insert(1, '../src/')

from Basic import *
from entrez_conversion import *
from pubmed_lib import *
from biopax_lib import *
from gemini_lib import *

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

email = "flalix@gmail.com"

# !pip3 install pyyaml
with open('params.yml', 'r') as file:
    dic_yml = yaml.safe_load(file)

#print(dic_yml)

In [ ]:
root_chibe = dic_yml['root_chibe']
root_colab = dic_yml['root_colab']
root0 = dic_yml['root0']

project = dic_yml['project']
s_project = dic_yml['s_project']

gene_protein = dic_yml['gene_protein']
s_omics = dic_yml['s_omics']

has_age = dic_yml['has_age']
has_gender = dic_yml['has_gender']

want_normalized = dic_yml['want_normalized']

abs_lfc_cutoff_inf = dic_yml['abs_lfc_cutoff_inf']
s_pathw_enrichm_method = dic_yml['s_pathw_enrichm_method']
num_min_degs_for_ptw_enr = dic_yml['num_min_degs_for_ptw_enr']
tolerance_pathway_index = dic_yml['tolerance_pathway_index']

case_list = dic_yml['case_list']
case_sel_list = dic_yml['case_sel_list']
s_len_case = dic_yml['s_len_case']

pval_pathway_cutoff = dic_yml['pval_pathway_cutoff']
fdr_pathway_cutoff = dic_yml['fdr_pathway_cutoff']
num_of_genes_cutoff = dic_yml['num_of_genes_cutoff']

run_list = dic_yml['run_list']
chosen_model_list = dic_yml['chosen_model_list']
i_dfp_list = dic_yml['i_dfp_list']

exp_normalization='quantile_norm' if want_normalized else None
normalization='not_normalized' if exp_normalization is None else exp_normalization

cfg = Config(project, s_project, case_list, root0)

case = case_list[0]

n_genes_annot_ptw, n_degs, n_degs_in_ptw, n_degs_not_in_ptw, degs_in_all_ratio = -1,-1,-1,-1,-1
abs_lfc_cutoff, fdr_lfc_cutoff, n_degs, n_degs_up, n_degs_dw = cfg.get_best_lfc_cutoff(case, 'not_normalized')


print(f"G/P LFC cutoffs: lfc={abs_lfc_cutoff:.3f}; fdr={fdr_lfc_cutoff:.3f}")
print(f"Pathway cutoffs: pval={pval_pathway_cutoff:.3f}; fdr={fdr_pathway_cutoff:.3f}; num of genes={num_of_genes_cutoff}")

In [ ]:
bpx = Biopax(gene_protein, s_omics, project, s_project, root0,
             case_list, has_age, has_gender, clone_objects=False,
             exp_normalization=exp_normalization, geneset_num=0, 
             num_min_degs_for_ptw_enr=num_min_degs_for_ptw_enr, 
             tolerance_pathway_index=tolerance_pathway_index, 
             s_pathw_enrichm_method = s_pathw_enrichm_method)

case = case_list[0]

bpx.cfg.set_default_best_lfc_cutoff(normalization, abs_lfc_cutoff=1, fdr_lfc_cutoff=0.05)
ret, degs, degs_ensembl, dfdegs = bpx.open_case(case, verbose=False)
print("\nEcho Parameters:")
bpx.echo_parameters()


### Ensemble: is_seldata=False

In [ ]:
###################
is_seldata=False

with_gender=bpx.has_gender
with_gender_list = [False, True] if with_gender else [False]

print(f"with_gender = {with_gender} because has_gender = {bpx.has_gender}")
###################

## Instantiating Gemini

In [ ]:
API_KEY = dic_yml['API_KEY']

disease = dic_yml['disease']
context_disease = dic_yml['context_disease']
n_sentences = dic_yml['n_sentences']
chosen_model_sampling = dic_yml['chosen_model_sampling']

gem = Gemini(bpx=bpx, is_seldata=is_seldata, disease=disease, context_disease=context_disease, 
             API_KEY=API_KEY, n_sentences=n_sentences, root0=root0, 
             chosen_model_list=chosen_model_list, i_dfp_list=i_dfp_list, chosen_model_sampling=chosen_model_sampling)
print("\n")
print(gem.disease, gem.is_seldata)
print("Context:", context_disease)

In [ ]:
is_seldata, case_list, s_len_case, run_list, chosen_model_list, i_dfp_list, chosen_model_sampling

In [ ]:
gem.set_case(bpx.case, bpx.df_enr, bpx.df_enr0)

In [ ]:
gem.is_seldata, gem.bpx.case_list, gem.chosen_model_list, gem.i_dfp_list, gem.chosen_model_sampling

In [ ]:
verbose=True

run='run01'
case=case_list[0]
print(">>>", case, '\n')

dfpiva = gem.open_gemini_dfpiva_all_models_one_run(run=run, chosen_model_list=chosen_model_list, verbose=verbose)
dfpiv = dfpiva[(dfpiva.case == case) & (dfpiva.i_dfp == 0)]

dfpiv.consensus.unique()

## 1) Statistics Default x BCA: confusion table

In [ ]:
verbose=True
force=False

run='run01'
case = case_list[0]
# group_discovery_fp_fn_enriched_bca
df, conf_list = gem.confusion_table_fp_fn_enriched_bca(run=run, case=case, chosen_model_list=chosen_model_list, force=force, verbose=verbose)

print(case, conf_list, '\n')
print(len(df))
df.head(3)

In [ ]:
df.groupby('group').count().reset_index().iloc[:,:2]

In [ ]:
verbose=False
force=False

run='run01'

for case in case_list:
    df, conf_list = gem.confusion_table_fp_fn_enriched_bca(run=run, case=case, chosen_model_list=chosen_model_list, force=force, verbose=verbose)

print(case, conf_list, '\n')
print(len(df))
df.head(3)

In [ ]:
df.groupby('group').count().reset_index().iloc[:,:2]

### stats

In [ ]:
force=False
verbose=True
prompt=False
param_perc=0.9

# dfa = gem.calc_all_significance_per_run_case_enriched_bca(run=run, case_list=case_list, chosen_model_list=chosen_model_list, 
dfa = gem.calc_confusion_stats_enriched_bca_per_run_case(run=run, case_list=case_list, chosen_model_list=chosen_model_list, 
                                                         param_perc=param_perc, prompt=prompt, force=force, verbose=verbose)
dfa

## 2) Statistics G0 x G1: confusion table
  - Positive control: i_dfp==0
  - fuzzy negative1: i_dfp==1
  - fuzzy negative2: i_dfp==2
  - negative control: i_dfp==3
  - calc:
    - TP, FP: positive control
    - TN1, FN1: fuzzy negative1
    - TN2, FN2: fuzzy negative2
    - TN, FN: negative control
    - calc: Sensitivity, Specificity, Accuracy, and F1-score

In [ ]:
verbose=True
force=False

dfconf1, dfconf2, dfconf3 = \
    gem.calc_gemini_4groups_confusion_table(run_list=run_list, case_list=case_list,
                                            chosen_model_list=chosen_model_list,
                                            force=force, verbose=verbose)

if dfconf1 is None:
    print("Could not calcualte")
    df3 = pd.DataFrame()
else:
    cols = ['case', 'which', 'n', 'npos', 'nneg', 'TP', 'FP', 'TN', 'FN', 'sens', 'spec', 'accu', 'prec', 'f1_score']
    
    run = run_list[0]
    
    df1 = dfconf1[dfconf1.run == run][cols]
    df2 = dfconf2[dfconf2.run == run][cols]
    df3 = dfconf3[dfconf3.run == run][cols]
    
    pd.options.display.float_format = '{:,.3f}'.format

    print("Ok")
  

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format

print(">>>", run, '\n')
mu_f1 = df1.f1_score.mean()
std_f1 = df1.f1_score.std()
print(f"G0 x G1: mu_f1 {100*mu_f1:.1f}% ({100*std_f1:.1f}%)\n")
df1

In [ ]:
print(">>>", run, '\n')
mu_f1 = df2.f1_score.mean()
std_f1 = df2.f1_score.std()
print(f"G0 x G2: mu_f1 {100*mu_f1:.1f}% ({100*std_f1:.1f}%)\n")
df2

In [ ]:
print(">>>", run, '\n')
mu_f3 = df3.f1_score.mean()
std_f3 = df3.f1_score.std()
print(f"G0 x G3: mu_f3 {100*mu_f3:.1f}% ({100*std_f3:.1f}%)\n")
df3

In [ ]:
mat = [('sens', 'Sensitivity'), ('spec', 'Specificity'), ('accu', 'Accuracy'),
	   ('prec', 'Precision'), ('f1_score', 'F1-score')]

for i in range(3):
    if i==0:
        dfa = df1
        dfb = df2
        compare = 'G0xG1 x G0xG2'
    elif i==1:
        dfa = df1
        dfb = df3
        compare = 'G0xG1 x G0xG3'
    else:
        dfa = df2
        dfb = df3
        compare = 'G0xG2 x G0xG3'

    for col, test in mat:
        # print(dfa[col])
        # print(dfb[col])
        
        mua  = dfa[col].mean()
        stda = dfa[col].std()

        mub  = dfb[col].mean()
        stdb = dfb[col].std()
        
        s_stat, stat, pval = calc_ttest(dfa[col], dfb[col])

        if stat is None:
            print(f"{compare}: {s_stat}")
        else:
            print(f"{compare}: {100*mua:.1f}% ({100*stda:.1f}%) x {100*mub:.1f}% ({100*stdb:.1f}%) -> {test:12} pval {pval:.2e}")
    print("")

In [ ]:
s_stat, stat, pval = calc_ttest(df1[col], df3[col])
s_stat

In [ ]:
s_stat, stat, pval = calc_ttest(df2[col], df3[col])
s_stat

In [ ]:
cols = ['run', 'which', 'case', 'n', 'npos', 'nneg', 'TP', 'FP', 'TN', 'FN', 
        'sens', 'spec', 'accu', 'prec', 'f1_score']

cols = ['run', 'which', 'case', 'n', 'npos', 'nneg', 'TP', 'FP', 'TN', 'FN', 
        'sens', 'spec', 'accu', 'prec', 'f1_score']


In [ ]:
run=run_list[1]

df11 = dfconf1[dfconf1.run == run][cols]
df12 = dfconf2[dfconf2.run == run][cols]
df13 = dfconf3[dfconf3.run == run][cols]

print(">>>", run, '\n')
df13

In [ ]:
df2

In [ ]:
df1

In [ ]:
run='run01'
verbose=True
force=True

df, df1, df2, df3 = gem.calc_gemini_4groups_confusion_stats(run=run, run_list=run_list,
                                                            case_list=case_list, chosen_model_list=chosen_model_list,
                                                            alpha=0.05, force=force, verbose=verbose)

print(len(df))
df.head(2)

In [ ]:
col = 'sens'
compare = 'G1xG0'
df[(df.run == run) & (df.test == col)   ] #  & (df['compare'] == compare)

In [ ]:
show_bar_errors=True

fig = gem.barplot_comparing_confusion_groups(run=run, run_list=run_list, case_list=case_list,
                                       chosen_model_list=chosen_model_list,
                                       width=1100, height=500, 
                                       fontsize=14, fontcolor='black',
                                       margin=dict( l=20, r=20, b=100, t=120, pad=4),
                                       plot_bgcolor="whitesmoke",
                                       xaxis_title="parameters", yaxis_title='percentage (%)',
                                       minus_test=-.2, minus_group=-0.1, 
                                       annot_fontfamily="Arial, monospace", annot_fontsize=12, 
                                       annot_fontcolor='black', show_bar_errors=show_bar_errors,
                                       savePlot=True, force=force, verbose=verbose)

fig.show()

In [ ]:
print(len(df))

# cols = ['run', 'test', 'test_desc', 'compare', 'n', 'mu', 'std', 'error', 'SEM', 'cinf', 'csup', 'pval', 'asteristics', 'stat', 's_stat']
cols = ['test_desc', 'compare', 'n', 'mu', 'std', 'error', 'SEM', 'cinf', 'csup', 'pval', 'pval_bonf', 'asteristics', 'stat', 's_stat']
df[df.test_desc == 'Sensitivity'][cols]

In [ ]:
# df.columns

### Comparing runs: stats

In [ ]:
verbose=True
force=False

dfstat = gem.calc_stats_gemini_4groups_confusion_compare_runs(run_list=run_list, case_list=case_list,
                                                               chosen_model_list=chosen_model_list,
                                                               force=force, verbose=verbose)

# cols = ['run0', 'run1', 'which', 'test', 'test_ext', 'mu_param0', 'std_param0', 'mu_param1', 'std_param1', 'stat_test', 'pvalue', 'stat', 'text_stat', 'text_ext']
dfstat.test_ext.unique()

In [ ]:
cols = ['which', 'test_ext', 'mu_param0', 'std_param0', 'mu_param1', 'std_param1', 'stat_test', 'pvalue', 'stat', 'text_ext']
tests = ['Sensitivity', 'Specificity', 'Accuracy', 'Precision', 'F1-score']

In [ ]:
dfstat[dfstat.test_ext == tests[0]][cols]

In [ ]:
dfstat[dfstat.test_ext == tests[1]][cols]

In [ ]:
dfstat[dfstat.test_ext == tests[2]][cols]

In [ ]:
dfstat[dfstat.test_ext == tests[3]][cols]

In [ ]:
dfstat[dfstat.test_ext == tests[4]][cols]

In [ ]:
run='run01'
case='WNT'
df, conf_list = gem.confusion_table_fp_fn_enriched_bca(run=run, case=case, chosen_model_list=chosen_model_list, force=force, verbose=verbose)
df

In [ ]:
tp, fp, tn, fn = conf_list
tp, fp, tn, fn

In [ ]:
dfpiva = gem.open_gemini_dfpiva_all_models_one_run(run=run, chosen_model_list=chosen_model_list, 
                                                    verbose=verbose)
dfpiva.i_dfp.unique()

## Development & tests

In [ ]:
force=False
verbose=False
param_perc=0.9

prompt=False

run='run01'
print(f"run {run} (chi-square statistics) param_perc={param_perc}:")

dic={}; icount=-1
for case in case_list:
    df, s_stat, stat, pvalue, dof, expected, list_obs, list_exp, Npos, Nneg = \
    gem.calc_significance_per_run_case_enriched_bca(run=run, case=case, chosen_model_list=chosen_model_list, 
                                                    param_perc=param_perc, force=force, verbose=verbose)

    if prompt:
        expected = list([ list(np.round(x,1)) for x in expected])
        print(f"\t{case:15} stat {str(np.round(stat,2)):7} pval {pvalue:.2e} - {str(list_obs):8} x {str(list_exp):8}; exp: {expected}")

    icount += 1
    dic[icount] = {}
    dic2 = dic[icount]

    dic2['run'] = run
    dic2['case']  = case

    dic2['TP'] = list_obs[0]
    dic2['FP'] = list_obs[1]
    dic2['TN'] = list_obs[2]
    dic2['FN'] = list_obs[3]

    dic2['TP_exp'] = list_exp[0]
    dic2['FP_exp'] = list_exp[1]
    dic2['TN_exp'] = list_exp[2]
    dic2['FN_exp'] = list_exp[3]

    dic2['stat'] = stat
    dic2['pvalue'] = pvalue
    dic2['dof'] = dof
    dic2['expected'] = np.round(expected, 2)

dfa = pd.DataFrame(dic).T
dfa['fdr'] = fdr(dfa.pvalue)

# cols = ['run', 'case', 'TP', 'FP', 'TN', 'FN', 'TP_exp', 'FP_exp', 'TN_exp', 'FN_exp', 'stat', 'pvalue', 'dof', 'expected', 'fdr']
cols = ['run', 'case', 'fdr', 'pvalue', 'TP', 'FP', 'TN', 'FN', 'TP_exp', 'FP_exp', 'TN_exp', 'FN_exp', 'expected']
dfa[cols]

In [ ]:
def barplot_comparing_groups(run:str, df1:pd.DataFrame, df2:pd.DataFrame, df3:pd.DataFrame, 
                             width:int=1100, height:int=600, 
                             fontsize:int=14, fontcolor:str='black',
                             margin:dict=dict( l=20, r=20, b=100, t=120, pad=4), plot_bgcolor:str="whitesmoke",
                             xaxis_title:str="parameters", yaxis_title:str='percentage (%)',
                             minus_test:float=-.1, minus_group:float=-0.05, 
                             annot_fontfamily:str="Arial, monospace", annot_fontsize:int=12, 
                             annot_fontcolor:str='black', savePlot:bool=True, alpha=0.05, verbose:bool=False):
    

    cols = ['case', 'which', 'n', 'npos', 'nneg', 'TP', 'FP', 'TN', 'FN', 'sens', 'spec', 'accu', 'prec', 'f1_score']
    
    df1 = dfconf1[dfconf1.run == run][cols]
    df2 = dfconf2[dfconf2.run == run][cols]
    df3 = dfconf3[dfconf3.run == run][cols]


    fig = go.Figure()
    
    title=f"Statistical parameters for Gi x G0 run {run}"
    x_num = 0

    mat = [('sens', 'Sensitivity'), ('spec', 'Specificity'), ('accu', 'Accuracy'),
    	   ('prec', 'Precision'), ('f1_score', 'F1-score')]
    # "aliceblue", "azure",
    colors_blue = ["blue", "blueviolet", "navy", "darkblue", "darkcyan", ]
    
    x_num=0
    dic={}; icount=-1
    for col, test in mat:

        vals_G0G3 = df3[col]
        
        for i in range(3):
            if i==0:
                dfa = df1
                compare = 'G1xG0'
            elif i==1:
                dfa = df2
                compare = 'G2xG0'
            else:
                dfa = df3
                compare = 'G3xG0'

            na = len(dfa)
            mua  = dfa[col].mean()
            stda = dfa[col].std()

            # alpha/2 Bonferroni correction, 2 comparisons.
            error, cinf, csup, SEM, stri =calc_confidence_interval_param(mua, stda, na, alpha=alpha/2, two_tailed=True)
            
            icount += 1
            dic[icount] = {}
            dic2 = dic[icount]

            dic2['run'] = run
            dic2['test'] = col
            dic2['test_desc'] = test
            dic2['compare'] = compare
            dic2['n'] = na
            dic2['mu'] = mua
            dic2['std'] = stda
            dic2['error'] = error
            dic2['SEM'] = SEM
            dic2['cinf'] = cinf
            dic2['csup'] = csup
            
            x_num += 1
            label = f"{compare}-{col}"
            fig.add_trace(go.Bar(x=[x_num], y=[mua], marker_color=colors_blue[i],
                                 error_y=dict(type='data', array=[error]), name=label))

            fig.add_annotation(
                x=x_num,
                y=minus_group,
                xref="x",
                yref="y",
                text=compare,
                showarrow=False,
                font=dict(
                    family="Arial, monospace",
                    size=fontsize,
                    color=fontcolor
                    ),
                align="center", )
            
            if i == 2:
                fig.add_annotation(
                    x=x_num-1,
                    y=minus_test,
                    xref="x",
                    yref="y",
                    text=test,
                    showarrow=False,
                    font=dict(
                        family="Arial, monospace",
                        size=fontsize,
                        color=fontcolor
                        ),
                    align="center", )

                x_num+=1

                dic2['pval'] = None
                dic2['asteristics'] = None
                dic2['stat'] = None
                dic2['s_stat'] = None

            else:
                s_stat, stat, pval = calc_ttest(dfa[col], vals_G0G3)
                aster = stat_asteristics(pval,NS='--')

                dic2['pval'] = pval
                dic2['asteristics'] = aster
                dic2['stat'] = stat
                dic2['s_stat'] = s_stat
                
                fig.add_annotation(
                    x=x_num,
                    y=mua+error+0.05,
                    xref="x",
                    yref="y",
                    text=f"{aster}", # <br>{pval:.2e}
                    showarrow=False,
                    font=dict(
                        family="Arial, monospace",
                        size=fontsize,
                        color=fontcolor
                        ),
                    align="center", )
                    
    fig.update_layout(
                autosize=True,
                title=title,
                width=width,
                height=height,
                plot_bgcolor=plot_bgcolor,
                xaxis_title=xaxis_title,
                yaxis_title=yaxis_title,
                xaxis_showticklabels=False,
                # yaxis_range=[0, 1],
                showlegend=False,
                font=dict(
                    family="Arial",
                    size=14,
                    color="Black"
                ),
    )

    df = pd.DataFrame(dic).T

    return fig, df

run='run01'
verbose=False

dfconf1, dfconf2, dfconf3 = \
    gem.calc_gemini_4groups_confusion_table(run_list=run_list, case_list=case_list,
                                            chosen_model_list=chosen_model_list,
                                            force=force, verbose=verbose)

fig, df = barplot_comparing_groups(run=run, df1=dfconf1, df2=dfconf2, df3=dfconf3, 
                                   width=1100, height=600, 
                                   fontsize=14, fontcolor='black',
                                   margin=dict( l=20, r=20, b=100, t=120, pad=4),
                                   plot_bgcolor="whitesmoke",
                                   xaxis_title="parameters", yaxis_title='percentage (%)',
                                   minus_test=-.1, minus_group=-0.05, 
                                   annot_fontfamily="Arial, monospace", annot_fontsize=12, 
                                   annot_fontcolor='black', savePlot=True, verbose=False)

fig.show()
df